In [2]:
import pandas as pd
import numpy as np

In [53]:
df_link_id = pd.read_csv('link_id.csv', dtype={'ROUTE_ID': str})
df_visum_link = pd.read_csv('visum_link.csv')
df_visum_stop = pd.read_csv('visum_stop.csv')

df_columns = ['$LINEROUTEITEM:LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE', 'INDEX', 'ISROUTEPOINT', 'NODENO', 'STOPPOINTNO', 'POSTLENGTH', 'ADDVAL']
df_line_route_items = pd.DataFrame(columns=df_columns)
df_line_route_items.to_csv('line_route_items.csv', index=False, encoding='utf-8-sig')

DIRECTIONCODE = '>'
POSTLENGTH = '0km'
ADDVAL = 0

i = 1
INDEX = 1
START_NODE = -1
NEXT_NODE = -1
prev_route_id = -1
prev_stop = -1
prev_link = -1
prev_link_no = -1

# df_link_id = df_link_id[(df_link_id['ROUTE_NAME'] == '1100') | (df_link_id['ROUTE_NAME'] == '1101')]
# df_link_id = df_link_id[(df_link_id['ROUTE_NAME'] == '1') | (df_link_id['ROUTE_NAME'] == '1100') | (df_link_id['ROUTE_NAME'] == '1101')]
for _, l in df_link_id.iterrows():
    print(i, '/', len(df_link_id), '(', i/len(df_link_id)*100, '%)')
    i = i + 1
    
    route_id = l['ROUTE_ID']
    route_name = l['ROUTE_NAME']
    link_id = l['LINK_ID']
    link_idx = l['INDEX']
    
    if prev_route_id != route_id:
        df_line_route_items.to_csv('line_route_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
        df_line_route_items = pd.DataFrame(columns=df_columns)
        INDEX = 1
        START_NODE = -1
        NEXT_NODE = -1
        prev_stop = -1
        prev_link = -1
        prev_link_no = -1
    
    prev_route_id = route_id
    
    if prev_link == link_id:
        continue
    
    prev_link = link_id
    
    if INDEX == 1:
        if len(set(df_visum_link[(df_visum_link['LINK_ID'] == link_id)]['LINK_NO'])) > 1:
            df_vlink = df_visum_link[df_visum_link['LINK_ID'] == link_id]
        
            for _, vl in df_vlink.iterrows():
                link_no = vl['LINK_NO']
                
                if prev_link_no == link_no:
                    continue
                
                prev_link_no = link_no

                df_vstop = df_visum_stop[df_visum_stop['LINK_NO'] == link_no]
                df_vstop = df_vstop.sort_values(by=['INDEX', 'F_NODE'])

                for _, vs in df_vstop.iterrows():
                    if prev_stop == vs['STOP_NO']:
                        continue

                    prev_stop = vs['STOP_NO']
                    
                    if INDEX > 1:
                        df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, NEXT_NODE, '', POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                        INDEX = INDEX + 1

                    df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                    INDEX = INDEX + 1
                    START_NODE = vs['F_NODE']
                    NEXT_NODE = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == START_NODE) & (df_visum_link['LINK_NO'] == link_no)].T_NODE.item()
        else:
            df_vlink = df_visum_link[df_visum_link['LINK_ID'] == link_id]
        
            for _, vl in df_vlink.iterrows():
                link_no = vl['LINK_NO']
                
                if prev_link_no == link_no:
                    continue
                
                prev_link_no = link_no

                df_vstop = df_visum_stop[df_visum_stop['LINK_NO'] == link_no]
                df_vstop = df_vstop.sort_values(by=['INDEX', 'F_NODE'])

                for _, vs in df_vstop.iterrows():
                    if prev_stop == vs['STOP_NO']:
                        continue

                    prev_stop = vs['STOP_NO']

                    df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                    INDEX = INDEX + 1
                    START_NODE = vs['F_NODE']
                    NEXT_NODE = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == START_NODE) & (df_visum_link['LINK_NO'] == link_no)].T_NODE.item()
    else:
        df_vlink = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == NEXT_NODE)]
        
        for _, vl in df_vlink.iterrows():
            link_no = vl['LINK_NO']
            
            if prev_link_no == link_no:
                continue
            
            prev_link_no = link_no
            
            df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, NEXT_NODE, '', POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
            INDEX = INDEX + 1
            
            NEXT_NODE = vl['T_NODE']
            
            df_vstop = df_visum_stop[df_visum_stop['LINK_NO'] == link_no]
            df_vstop = df_vstop.sort_values(by=['INDEX', 'F_NODE'])

            for _, vs in df_vstop.iterrows():
                if prev_stop == vs['STOP_NO']:
                    continue
                
                prev_stop = vs['STOP_NO']
                
                df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                INDEX = INDEX + 1
                
        if len(set(df_visum_link[(df_visum_link['LINK_ID'] == link_id)]['LINK_NO'])) > 1:
            df_vlink = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == NEXT_NODE)]
        
            for _, vl in df_vlink.iterrows():
                link_no = vl['LINK_NO']

                if prev_link_no == link_no:
                    continue

                prev_link_no = link_no

                df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, NEXT_NODE, '', POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                INDEX = INDEX + 1

                NEXT_NODE = vl['T_NODE']

                df_vstop = df_visum_stop[df_visum_stop['LINK_NO'] == link_no]
                df_vstop = df_vstop.sort_values(by=['INDEX', 'F_NODE'])

                for _, vs in df_vstop.iterrows():
                    if prev_stop == vs['STOP_NO']:
                        continue

                    prev_stop = vs['STOP_NO']

                    df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                    INDEX = INDEX + 1

df_line_route_items.to_csv('line_route_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
df_line_route_items = pd.DataFrame(columns=df_columns)

1 / 55598 ( 0.001798625849850714 %)
2 / 55598 ( 0.003597251699701428 %)
3 / 55598 ( 0.005395877549552142 %)
4 / 55598 ( 0.007194503399402856 %)
5 / 55598 ( 0.00899312924925357 %)
6 / 55598 ( 0.010791755099104284 %)
7 / 55598 ( 0.012590380948954998 %)
8 / 55598 ( 0.014389006798805712 %)
9 / 55598 ( 0.016187632648656428 %)
10 / 55598 ( 0.01798625849850714 %)
11 / 55598 ( 0.019784884348357852 %)
12 / 55598 ( 0.021583510198208568 %)
13 / 55598 ( 0.023382136048059284 %)
14 / 55598 ( 0.025180761897909996 %)
15 / 55598 ( 0.02697938774776071 %)
16 / 55598 ( 0.028778013597611424 %)
17 / 55598 ( 0.03057663944746214 %)
18 / 55598 ( 0.032375265297312855 %)
19 / 55598 ( 0.034173891147163564 %)
20 / 55598 ( 0.03597251699701428 %)
21 / 55598 ( 0.037771142846864995 %)
22 / 55598 ( 0.039569768696715704 %)
23 / 55598 ( 0.04136839454656642 %)
24 / 55598 ( 0.043167020396417136 %)
25 / 55598 ( 0.04496564624626785 %)
26 / 55598 ( 0.04676427209611857 %)
27 / 55598 ( 0.04856289794596928 %)
28 / 55598 ( 0.0503

243 / 55598 ( 0.4370660815137235 %)
244 / 55598 ( 0.43886470736357425 %)
245 / 55598 ( 0.4406633332134249 %)
246 / 55598 ( 0.4424619590632757 %)
247 / 55598 ( 0.4442605849131264 %)
248 / 55598 ( 0.44605921076297705 %)
249 / 55598 ( 0.44785783661282785 %)
250 / 55598 ( 0.4496564624626785 %)
251 / 55598 ( 0.45145508831252923 %)
252 / 55598 ( 0.4532537141623799 %)
253 / 55598 ( 0.45505234001223066 %)
254 / 55598 ( 0.45685096586208135 %)
255 / 55598 ( 0.4586495917119321 %)
256 / 55598 ( 0.4604482175617828 %)
257 / 55598 ( 0.4622468434116335 %)
258 / 55598 ( 0.46404546926148427 %)
259 / 55598 ( 0.46584409511133495 %)
260 / 55598 ( 0.4676427209611857 %)
261 / 55598 ( 0.46944134681103633 %)
262 / 55598 ( 0.47123997266088713 %)
263 / 55598 ( 0.47303859851073776 %)
264 / 55598 ( 0.4748372243605885 %)
265 / 55598 ( 0.4766358502104392 %)
266 / 55598 ( 0.47843447606028994 %)
267 / 55598 ( 0.4802331019101406 %)
268 / 55598 ( 0.48203172775999137 %)
269 / 55598 ( 0.48383035360984206 %)
270 / 55598 ( 

503 / 55598 ( 0.9047088024749091 %)
504 / 55598 ( 0.9065074283247598 %)
505 / 55598 ( 0.9083060541746106 %)
506 / 55598 ( 0.9101046800244613 %)
507 / 55598 ( 0.911903305874312 %)
508 / 55598 ( 0.9137019317241627 %)
509 / 55598 ( 0.9155005575740135 %)
510 / 55598 ( 0.9172991834238642 %)
511 / 55598 ( 0.9190978092737149 %)
512 / 55598 ( 0.9208964351235656 %)
513 / 55598 ( 0.9226950609734164 %)
514 / 55598 ( 0.924493686823267 %)
515 / 55598 ( 0.9262923126731177 %)
516 / 55598 ( 0.9280909385229685 %)
517 / 55598 ( 0.9298895643728192 %)
518 / 55598 ( 0.9316881902226699 %)
519 / 55598 ( 0.9334868160725205 %)
520 / 55598 ( 0.9352854419223714 %)
521 / 55598 ( 0.9370840677722221 %)
522 / 55598 ( 0.9388826936220727 %)
523 / 55598 ( 0.9406813194719233 %)
524 / 55598 ( 0.9424799453217743 %)
525 / 55598 ( 0.9442785711716248 %)
526 / 55598 ( 0.9460771970214755 %)
527 / 55598 ( 0.9478758228713264 %)
528 / 55598 ( 0.949674448721177 %)
529 / 55598 ( 0.9514730745710277 %)
530 / 55598 ( 0.953271700420878

732 / 55598 ( 1.3165941220907227 %)
733 / 55598 ( 1.3183927479405733 %)
734 / 55598 ( 1.320191373790424 %)
735 / 55598 ( 1.3219899996402749 %)
736 / 55598 ( 1.3237886254901254 %)
737 / 55598 ( 1.3255872513399762 %)
738 / 55598 ( 1.327385877189827 %)
739 / 55598 ( 1.3291845030396776 %)
740 / 55598 ( 1.3309831288895284 %)
741 / 55598 ( 1.332781754739379 %)
742 / 55598 ( 1.3345803805892298 %)
743 / 55598 ( 1.3363790064390806 %)
744 / 55598 ( 1.3381776322889312 %)
745 / 55598 ( 1.339976258138782 %)
746 / 55598 ( 1.3417748839886328 %)
747 / 55598 ( 1.3435735098384833 %)
748 / 55598 ( 1.3453721356883341 %)
749 / 55598 ( 1.347170761538185 %)
750 / 55598 ( 1.3489693873880355 %)
751 / 55598 ( 1.3507680132378863 %)
752 / 55598 ( 1.3525666390877369 %)
753 / 55598 ( 1.3543652649375877 %)
754 / 55598 ( 1.3561638907874385 %)
755 / 55598 ( 1.357962516637289 %)
756 / 55598 ( 1.3597611424871396 %)
757 / 55598 ( 1.3615597683369907 %)
758 / 55598 ( 1.3633583941868412 %)
759 / 55598 ( 1.3651570200366918 %

965 / 55598 ( 1.735673945105939 %)
966 / 55598 ( 1.7374725709557899 %)
967 / 55598 ( 1.7392711968056405 %)
968 / 55598 ( 1.7410698226554913 %)
969 / 55598 ( 1.742868448505342 %)
970 / 55598 ( 1.7446670743551926 %)
971 / 55598 ( 1.7464657002050434 %)
972 / 55598 ( 1.748264326054894 %)
973 / 55598 ( 1.7500629519047448 %)
974 / 55598 ( 1.7518615777545956 %)
975 / 55598 ( 1.7536602036044462 %)
976 / 55598 ( 1.755458829454297 %)
977 / 55598 ( 1.7572574553041478 %)
978 / 55598 ( 1.7590560811539984 %)
979 / 55598 ( 1.7608547070038492 %)
980 / 55598 ( 1.7626533328536995 %)
981 / 55598 ( 1.7644519587035505 %)
982 / 55598 ( 1.7662505845534013 %)
983 / 55598 ( 1.7680492104032517 %)
984 / 55598 ( 1.7698478362531027 %)
985 / 55598 ( 1.7716464621029535 %)
986 / 55598 ( 1.7734450879528039 %)
987 / 55598 ( 1.7752437138026549 %)
988 / 55598 ( 1.7770423396525057 %)
989 / 55598 ( 1.778840965502356 %)
990 / 55598 ( 1.780639591352207 %)
991 / 55598 ( 1.7824382172020574 %)
992 / 55598 ( 1.7842368430519082 %

...

54518 / 55598 ( 98.05748408216122 %)
54519 / 55598 ( 98.05928270801107 %)
54520 / 55598 ( 98.06108133386093 %)
54521 / 55598 ( 98.06287995971078 %)
54522 / 55598 ( 98.06467858556063 %)
54523 / 55598 ( 98.06647721141049 %)
54524 / 55598 ( 98.06827583726033 %)
54525 / 55598 ( 98.07007446311017 %)
54526 / 55598 ( 98.07187308896003 %)
54527 / 55598 ( 98.07367171480989 %)
54528 / 55598 ( 98.07547034065973 %)
54529 / 55598 ( 98.07726896650959 %)
54530 / 55598 ( 98.07906759235944 %)
54531 / 55598 ( 98.08086621820928 %)
54532 / 55598 ( 98.08266484405914 %)
54533 / 55598 ( 98.08446346990898 %)
54534 / 55598 ( 98.08626209575884 %)
54535 / 55598 ( 98.0880607216087 %)
54536 / 55598 ( 98.08985934745854 %)
54537 / 55598 ( 98.09165797330839 %)
54538 / 55598 ( 98.09345659915824 %)
54539 / 55598 ( 98.09525522500809 %)
54540 / 55598 ( 98.09705385085795 %)
54541 / 55598 ( 98.0988524767078 %)
54542 / 55598 ( 98.10065110255765 %)
54543 / 55598 ( 98.10244972840749 %)
54544 / 55598 ( 98.10424835425735 %)
545

54745 / 55598 ( 98.46577215007734 %)
54746 / 55598 ( 98.4675707759272 %)
54747 / 55598 ( 98.46936940177704 %)
54748 / 55598 ( 98.4711680276269 %)
54749 / 55598 ( 98.47296665347675 %)
54750 / 55598 ( 98.47476527932659 %)
54751 / 55598 ( 98.47656390517643 %)
54752 / 55598 ( 98.47836253102629 %)
54753 / 55598 ( 98.48016115687615 %)
54754 / 55598 ( 98.481959782726 %)
54755 / 55598 ( 98.48375840857585 %)
54756 / 55598 ( 98.4855570344257 %)
54757 / 55598 ( 98.48735566027554 %)
54758 / 55598 ( 98.4891542861254 %)
54759 / 55598 ( 98.49095291197526 %)
54760 / 55598 ( 98.4927515378251 %)
54761 / 55598 ( 98.49455016367496 %)
54762 / 55598 ( 98.4963487895248 %)
54763 / 55598 ( 98.49814741537465 %)
54764 / 55598 ( 98.4999460412245 %)
54765 / 55598 ( 98.50174466707435 %)
54766 / 55598 ( 98.5035432929242 %)
54767 / 55598 ( 98.50534191877406 %)
54768 / 55598 ( 98.50714054462391 %)
54769 / 55598 ( 98.50893917047375 %)
54770 / 55598 ( 98.51073779632361 %)
54771 / 55598 ( 98.51253642217345 %)
54772 / 555

54983 / 55598 ( 98.89384510234181 %)
54984 / 55598 ( 98.89564372819166 %)
54985 / 55598 ( 98.89744235404152 %)
54986 / 55598 ( 98.89924097989137 %)
54987 / 55598 ( 98.90103960574122 %)
54988 / 55598 ( 98.90283823159106 %)
54989 / 55598 ( 98.90463685744092 %)
54990 / 55598 ( 98.90643548329076 %)
54991 / 55598 ( 98.90823410914061 %)
54992 / 55598 ( 98.91003273499047 %)
54993 / 55598 ( 98.91183136084032 %)
54994 / 55598 ( 98.91362998669017 %)
54995 / 55598 ( 98.91542861254003 %)
54996 / 55598 ( 98.91722723838987 %)
54997 / 55598 ( 98.91902586423971 %)
54998 / 55598 ( 98.92082449008957 %)
54999 / 55598 ( 98.92262311593942 %)
55000 / 55598 ( 98.92442174178927 %)
55001 / 55598 ( 98.92622036763913 %)
55002 / 55598 ( 98.92801899348898 %)
55003 / 55598 ( 98.92981761933882 %)
55004 / 55598 ( 98.93161624518868 %)
55005 / 55598 ( 98.93341487103852 %)
55006 / 55598 ( 98.93521349688838 %)
55007 / 55598 ( 98.93701212273824 %)
55008 / 55598 ( 98.93881074858808 %)
55009 / 55598 ( 98.94060937443793 %)
5

55234 / 55598 ( 99.34530019065434 %)
55235 / 55598 ( 99.34709881650419 %)
55236 / 55598 ( 99.34889744235404 %)
55237 / 55598 ( 99.35069606820389 %)
55238 / 55598 ( 99.35249469405375 %)
55239 / 55598 ( 99.3542933199036 %)
55240 / 55598 ( 99.35609194575345 %)
55241 / 55598 ( 99.35789057160329 %)
55242 / 55598 ( 99.35968919745315 %)
55243 / 55598 ( 99.361487823303 %)
55244 / 55598 ( 99.36328644915284 %)
55245 / 55598 ( 99.3650850750027 %)
55246 / 55598 ( 99.36688370085255 %)
55247 / 55598 ( 99.3686823267024 %)
55248 / 55598 ( 99.37048095255226 %)
55249 / 55598 ( 99.3722795784021 %)
55250 / 55598 ( 99.37407820425194 %)
55251 / 55598 ( 99.3758768301018 %)
55252 / 55598 ( 99.37767545595165 %)
55253 / 55598 ( 99.3794740818015 %)
55254 / 55598 ( 99.38127270765136 %)
55255 / 55598 ( 99.3830713335012 %)
55256 / 55598 ( 99.38486995935105 %)
55257 / 55598 ( 99.38666858520091 %)
55258 / 55598 ( 99.38846721105075 %)
55259 / 55598 ( 99.39026583690061 %)
55260 / 55598 ( 99.39206446275047 %)
55261 / 55

55460 / 55598 ( 99.7517896327206 %)
55461 / 55598 ( 99.75358825857045 %)
55462 / 55598 ( 99.7553868844203 %)
55463 / 55598 ( 99.75718551027015 %)
55464 / 55598 ( 99.75898413612 %)
55465 / 55598 ( 99.76078276196986 %)
55466 / 55598 ( 99.76258138781971 %)
55467 / 55598 ( 99.76438001366955 %)
55468 / 55598 ( 99.76617863951941 %)
55469 / 55598 ( 99.76797726536925 %)
55470 / 55598 ( 99.7697758912191 %)
55471 / 55598 ( 99.77157451706896 %)
55472 / 55598 ( 99.77337314291881 %)
55473 / 55598 ( 99.77517176876866 %)
55474 / 55598 ( 99.77697039461852 %)
55475 / 55598 ( 99.77876902046836 %)
55476 / 55598 ( 99.7805676463182 %)
55477 / 55598 ( 99.78236627216806 %)
55478 / 55598 ( 99.7841648980179 %)
55479 / 55598 ( 99.78596352386776 %)
55480 / 55598 ( 99.78776214971762 %)
55481 / 55598 ( 99.78956077556747 %)
55482 / 55598 ( 99.79135940141731 %)
55483 / 55598 ( 99.79315802726717 %)
55484 / 55598 ( 99.79495665311701 %)
55485 / 55598 ( 99.79675527896687 %)
55486 / 55598 ( 99.79855390481673 %)
55487 / 5